# <font color=red>PREPARE DATA</font>
## 1. Read in a model field and the Z Field. 
## 2. convert Z field from MSL to AGL
## 3. interpolate the model field to a Z AGL level
## 4. scale the data to be from 0 to 1
## 5. break the data into smaller arrays

In [1]:
import xarray as xr
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import load_data
import wrf

In [2]:
def downscale(feature):
    
    a = np.zeros((feature.shape[0]*4,feature.shape[1],128,128))
    
    j = 0
    for i in range(feature.shape[0]):
        for v in range(feature.shape[1]):
            a[j,v,:,:] = feature[i,v,0:128,0:128]
            
            a[j+1,v,:,:] = feature[i,v,0:128,128:256]
 
            a[j+2,v,:,:] = feature[i,v,128:256,0:128]
        
            a[j+3,v,:,:] = feature[i,v,128:256,128:256]
            
        j = j + 4

    print("number of 128x128 images:", j)
    return a

def downscale_2(feature, label):
    
    a = np.zeros((feature.shape[0]*4,feature.shape[1],128,128))
    b = np.zeros((feature.shape[0]*4,feature.shape[1],128,128))
    
    j = 0
    for i in range(feature.shape[0]):
        for v in range(feature.shape[1]):
            a[j,v,:,:] = feature[i,v,0:128,0:128]
            b[j,v,:,:] = label[i,v,0:128,0:128]
            
            a[j+1,v,:,:] = feature[i,v,0:128,128:256]
            b[j+1,v,:,:] = label[i,v,0:128,128:256]
 
            a[j+2,v,:,:] = feature[i,v,128:256,0:128]
            b[j+2,v,:,:] = label[i,v,128:256,0:128]
        
            a[j+3,v,:,:] = feature[i,v,128:256,128:256]
            b[j+3,v,:,:] = label[i,v,128:256,128:256]
            
        j = j + 4

    print("number of 128x128 images:", j)
    return a,b

def downscale_remove(feature, label):
    
    a = np.zeros((feature.shape[0]*4,feature.shape[1],128,128))
    b = np.zeros((feature.shape[0]*4,feature.shape[1],128,128))
    
    j = 0
    for i in range(feature.shape[0]):
        for v in range(feature.shape[1]):
            if (np.amax(feature[i,v,0:128,0:128]) != np.amin(feature[i,v,0:128,0:128]) and
                np.amax(feature[i,v,0:128,128:256]) != np.amin(feature[i,v,0:128,128:256]) and
                np.amax(feature[i,v,128:256,0:128]) != np.amin(feature[i,v,128:256,0:128]) and
                np.amax(feature[i,v,128:256,128:256]) != np.amin(feature[i,v,128:256,128:256]) ):
                    a[j,v,:,:] = feature[i,v,0:128,0:128]
                    b[j,v,:,:] = label[i,v,0:128,0:128]
            
                    a[j+1,v,:,:] = feature[i,v,0:128,128:256]
                    b[j+1,v,:,:] = label[i,v,0:128,128:256]
 
                    a[j+2,v,:,:] = feature[i,v,128:256,0:128]
                    b[j+2,v,:,:] = label[i,v,128:256,0:128]
        
                    a[j+3,v,:,:] = feature[i,v,128:256,128:256]
                    b[j+3,v,:,:] = label[i,v,128:256,128:256]
                    
        j = j + 4
                   
                
    print("number of 128x128 images:", j)
    for i in range(j,feature.shape[0]):
        a = np.delete(a, j, 0)
        b = np.delete(b, j, 0)

    #return np.resize(a, (j,min_nv,128,128), np.resize(b, (j,min_nv,128,128))
    return a,b

def interp_data(fieldname, field, Z_AGL):
    
    #interp_levels = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000]
    interp_levels = [0, 500, 2000]
    
    vars = {}
    
    for level in interp_levels:

        varname = fieldname + "_" + str(level) + "m_AGL"
    
        if level == 0:
            vars[varname] = field[:,0,:,:]
            vars[varname] = vars[varname][:,np.newaxis,:,:]
        else:
            #
            # interpolate to the requested level
            # 
            vars[varname] = wrf.interplevel(field[:,:,:,:], Z_AGL[:,:,:,:], level, meta=False)
            vars[varname] = vars[varname][:, np.newaxis,:,:]
        
    return vars

def interp_data_concat(field, Z_AGL):
    
    #interp_levels = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000]
    interp_levels = [0, 500, 2000]
    
    count = 0
    for level in interp_levels:
        
        count = count + 1
        
        if level == 0:
            interp_field = field[:,0,:,:]
            interp_field = interp_field[:,np.newaxis,:,:]
        else:
            #
            # interpolate to the requested level
            # 
            interp_field = wrf.interplevel(field[:,:,:,:], Z_AGL[:,:,:,:], level, meta=False)
            interp_field = interp_field[:, np.newaxis,:,:]
        
        if count == 1:
            interp_3d = interp_field
            print(interp_3d.shape)
        else:
            interp_3d = np.concatenate( (interp_3d, interp_field), axis=1)
            print(interp_3d.shape)
                
    return interp_3d

In [3]:
#
# Load the Z data and get the terrain height using the 
# models lowest Z level (hybrid level data)
#

z = load_data.load_Z_data_oneTime()
#z = load_data.load_Z_data_all(slevel=0, elevel=50, method='sel')

terrain_height = z[:,0,:,:].copy()
terrain_height = terrain_height[:,np.newaxis,:,:]
print(z.shape)
print(terrain_height.shape)

(8, 50, 256, 256)
(8, 1, 256, 256)


In [4]:
#
# move the channels from position 1 ot position 3
# goes from [time,channel,height,width] to [time, height, width, channel]
# which is the default for Conv2D.
#
#z = np.moveaxis(z, 1, 3)
#print(z.shape)

In [5]:
#
# Convert the Z data from MSL to AGL
#

#for i in range(0,z.shape[1]):
#    z[:,i,:,:] = z[:,i,:,:] - terrain_height[:,0,:,:]

# this is the same as the line below but more efficient
z = z - terrain_height
print(z.shape)
del terrain_height

(8, 50, 256, 256)


In [6]:
w = load_data.load_W_data_oneTime()
qr = load_data.load_QRAIN_data_oneTime()

W_VAR = interp_data_concat(w, z)
del w

QR_VAR = interp_data_concat(qr, z)
del qr

(8, 1, 256, 256)
(8, 2, 256, 256)
(8, 3, 256, 256)
(8, 1, 256, 256)
(8, 2, 256, 256)
(8, 3, 256, 256)


In [7]:
#DS_feature, DS_label = downscale_2(QR_VAR, W_VAR)
DS_feature = downscale(QR_VAR)

number of 128x128 images: 32


In [9]:
print(DS_feature.shape)
#print(DS_label.shape)
for i in range(DS_feature.shape[0]):
    for v in range(DS_feature.shape[1]):
        print("i:", i, "v:",v)

        print (np.amin(DS_feature[i,v,:,:]))
        print (np.amax(DS_feature[i,v,:,:]))

(32, 3, 128, 128)
i: 0 v: 0
0.0
0.0019415419083088636
i: 0 v: 1
0.0
0.0021832252386957407
i: 0 v: 2
0.0
0.0019743647426366806
i: 1 v: 0
0.0
0.0027389060705900192
i: 1 v: 1
0.0
0.002946280175819993
i: 1 v: 2
0.0
0.0037056219298392534
i: 2 v: 0
0.0
0.0005428703152574599
i: 2 v: 1
0.0
0.0004966292763128877
i: 2 v: 2
0.0
0.0011659294832497835
i: 3 v: 0
0.0
0.0016709117917343974
i: 3 v: 1
0.0
0.001890540705062449
i: 3 v: 2
0.0
0.0016935366438701749
i: 4 v: 0
0.0
0.00032956074574030936
i: 4 v: 1
0.0
0.0004347909416537732
i: 4 v: 2
0.0
0.0008657117141410708
i: 5 v: 0
0.0
0.0015942261088639498
i: 5 v: 1
0.0
0.0017817417392507195
i: 5 v: 2
0.0
0.0014665080234408379
i: 6 v: 0
0.0
0.00042450864566490054
i: 6 v: 1
0.0
0.0004662575083784759
i: 6 v: 2
0.0
1.456955124012893e-05
i: 7 v: 0
0.0
0.0017049504676833749
i: 7 v: 1
0.0
0.0018548467196524143
i: 7 v: 2
0.0
0.002265625400468707
i: 8 v: 0
0.0
0.0007167785079218447
i: 8 v: 1
0.0
0.000903646694496274
i: 8 v: 2
0.0
0.0013494240120053291
i: 9 v: 0
0.

In [ ]:
#
# Load the model label field
#

w = load_data.load_W_data_oneTime()
#w = load_data.load_W_data_all(slevel=0, elevel=50, method="sel")
print(w.shape)

#
# interpolated the W field to the requested
# heights AGL
#
W_VAR = interp_data_concat(w, z)
del w

for i in range(W_VAR.shape[0] - 1):
    for v in range(W_VAR.shape[1] - 1):
        if np.amax(W_VAR[i,v,:,:]) == np.amin(W_VAR[i,v,:,:]):
            W_VAR[i,v,:,:] == 0
        else:
            W_VAR[i,v,:,:] = (W_VAR[i,v,:,:] - np.amin(W_VAR[i,v,:,:])) / (np.amax(W_VAR[i,v,:,:]) - np.amin(W_VAR[i,v,:,:]))

output_data = '/glade/work/hardt/ds612/test-2000-2013_June-Sept_W_INTERP_AGL.nc'
WAGL = xr.DataArray(W_VAR, name='W')
encoding={'W': {'zlib': True, '_FillValue': -99.0}}
WAGL.to_netcdf(output_data, encoding=encoding)

del W_VAR

In [ ]:
#
# Load the model feature field
#

qr = load_data.load_QRAIN_data_oneTime()
#qr = load_data.load_QRAIN_data_all(slevel=0, elevel=50, method="sel")
print(qr.shape)

#
# interpolated the QRAIN field to the requested
# heights AGL 
#
QR_VAR = interp_data_concat(qr, z)
del qr

for i in range(QR_VAR.shape[0] - 1):
    for v in range(QR_VAR.shape[1] - 1):
        if np.amax(QR_VAR[i,v,:,:]) == np.amin(QR_VAR[i,v,:,:]):
            QR_VAR[i,v,:,:] == 0
        else:
            QR_VAR[i,v,:,:] = (QR_VAR[i,v,:,:] - np.amin(QR_VAR[i,v,:,:])) / (np.amax(QR_VAR[i,v,:,:]) - np.amin(QR_VAR[i,v,:,:]))

output_data = '/glade/work/hardt/ds612/test-2000-2013_June-Sept_QRAIN_INTERP_AGL.nc'
QR = xr.DataArray(QR_VAR, name='QRAIN')
encoding={'QRAIN': {'zlib': True, '_FillValue': -99.0}}
QR.to_netcdf(output_data, encoding=encoding)

#del QR_VAR

In [ ]:
#
# Load another model feature field (QSNOW)
#

qs = load_data.load_QSNOW_data_oneTime()
#qs = load_data.load_QSNOW_data_all(slevel=0, elevel=50, method="sel")
print(qs.shape)

#
# interpolated the QSNOW field to the requested
# heights AGL
#
QS_VAR = interp_data_concat(qs, z)
del qs

for i in range(QS_VAR.shape[0] - 1):
    for v in range(QS_VAR.shape[1] - 1):
        if np.amax(QS_VAR[i,v,:,:]) == np.amin(QS_VAR[i,v,:,:]):
            QS_VAR[i,v,:,:] == 0
        else:
            QS_VAR[i,v,:,:] = (QS_VAR[i,v,:,:] - np.amin(QS_VAR[i,v,:,:])) / (np.amax(QS_VAR[i,v,:,:]) - np.amin(QS_VAR[i,v,:,:]))

output_data = '/glade/work/hardt/ds612/test-2000-2013_June-Sept_QSNOW_INTERP_AGL.nc'
QS = xr.DataArray(QS_VAR, name='QSNOW')
encoding={'QSNOW': {'zlib': True, '_FillValue': -99.0}}
QS.to_netcdf(output_data, encoding=encoding)

del QS_VAR

In [ ]:
import matplotlib.pyplot as plt

d1 = W_VAR[0,1,:,:]
d2 = QR_VAR[0,1,:,:]

Wp = np.percentile(d1,99.9)
Wmin = np.amin(d1)
Wmax = np.amax(d1)
print(Wmin, Wmax, Wp)

Qp = np.percentile(d2, 99.9)
Qmin = np.amin(d2)
Qmax = np.amax(d2)
print(Qmin, Qmax, Qp)

cmap = plt.cm.Spectral_r

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(36,6))

w5 = ax1.imshow(d1, cmap=cmap)
ax1.set_title("W at 500m AGL level")
#ax = plt.axes(projection=ccrs.PlateCarree())
#ax.set_global()
#ax.coastlines()
w5.set_clim(vmin=Wmin, vmax=Wmax)
fig.colorbar(w5, ax=ax1, orientation='vertical', label='')

qr5 = ax2.imshow(d2, cmap=cmap)
ax2.set_title("QRAIN at 500m AGL")
#ax = plt.axes(projection=ccrs.PlateCarree())
#ax.set_global()
#ax.coastlines()
qr5.set_clim(vmin=Qmin,vmax=Qmax)
fig.colorbar(qr5, ax=ax2, orientation='vertical', label='')


In [ ]:
for i in range(QR_VAR.shape[0] - 1):
    for z in range(QR_VAR.shape[1] - 1):
        QR_VAR[i,z,:,:] = (QR_VAR[i,z,:,:] - np.amin(QR_VAR[i,z,:,:])) / (np.amax(QR_VAR[i,z,:,:]) - np.amin(QR_VAR[i,z,:,:]))
        W_VAR[i,z,:,:] = (W_VAR[i,z,:,:] - np.amin(W_VAR[i,z,:,:])) / (np.amax(W_VAR[i,z,:,:]) - np.amin(W_VAR[i,z,:,:]))


In [ ]:
d1 = W_VAR[0,1,:,:]
d2 = QR_VAR[0,1,:,:]

Wp = np.percentile(d1,99.9)
Wmin = np.amin(d1)
Wmax = np.amax(d1)
print(Wmin, Wmax, Wp)

Qp = np.percentile(d2, 99.9)
Qmin = np.amin(d2)
Qmax = np.amax(d2)
print(Qmin, Qmax, Qp)

cmap = plt.cm.Spectral_r

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(36,6))

w5 = ax1.imshow(d1, cmap=cmap)
ax1.set_title("W at 500m AGL level")
#ax = plt.axes(projection=ccrs.PlateCarree())
#ax.set_global()
#ax.coastlines()
w5.set_clim(vmin=Wmin, vmax=Wmax)
fig.colorbar(w5, ax=ax1, orientation='vertical', label='')

qr5 = ax2.imshow(d2, cmap=cmap)
ax2.set_title("QRAIN at 500m AGL")
#ax = plt.axes(projection=ccrs.PlateCarree())
#ax.set_global()
#ax.coastlines()
qr5.set_clim(vmin=Qmin,vmax=Qmax)
fig.colorbar(qr5, ax=ax2, orientation='vertical', label='')


In [ ]:
output_data = '/glade/work/hardt/ds612/2000-2013_June-Sept_QR_INTERP_AGL.nc'
QR = xr.DataArray(QR_VAR, name='QRAIN')
encoding={'QRAIN': {'zlib': True, '_FillValue': -99.0}}
QR.to_netcdf(output_data, encoding=encoding)

output_data = '/glade/work/hardt/ds612/2000-2013_June-Sept_W_INTERP_AGL.nc'
WAGL = xr.DataArray(W_VAR, name='W')
encoding={'W': {'zlib': True, '_FillValue': -99.0}}
WAGL.to_netcdf(output_data, encoding=encoding)


In [ ]:
for W_key, QR_key in zip(W_VARS.keys(), QR_VARS.keys()):
    
    print(W_key, W_VARS[W_key].shape)
    print(QR_key, QR_VARS[QR_key].shape)
    
    DS_feature, DS_label = downscale_remove(QR_VARS[QR_key], W_VARS[W_key])

    #DS_feature = (DS_feature - np.amin(DS_feature)) / (np.amax(DS_feature) - np.amin(DS_feature))
    #DS_label = (DS_label - np.amin(DS_label)) / (np.amax(DS_label) - np.amin(DS_label))

    for i in range(len(DS_feature) - 1):
        DS_feature[i,:,:] = (DS_feature[i,:,:] - np.amin(DS_feature[i,:,:])) / (np.amax(DS_feature[i,:,:]) - np.amin(DS_feature[i,:,:]))
        DS_label[i,:,:] = (DS_label[i,:,:] - np.amin(DS_label[i,:,:])) / (np.amax(DS_label[i,:,:]) - np.amin(DS_label[i,:,:]))

    #
    # write downscaled feature_data
    #
    output_data = '/glade/work/hardt/ds612/2000-2013_June-Sept_DS128_'+ QR_key + '.nc'
    QR = xr.DataArray(DS_feature, name='QRAIN')
    encoding={'QRAIN': {'zlib': True, '_FillValue': -99.0}}
    #QR.to_netcdf(output_data, encoding=encoding)
    
    #
    # write downscaled label_data
    #
    output_data = '/glade/work/hardt/ds612/2000-2013_June-Sept_DS128_' + W_key + '.nc'
    WAGL = xr.DataArray(DS_label, name='W')
    encoding={'W': {'zlib': True, '_FillValue': -99.0}}
    #WAGL.to_netcdf(output_data, encoding=encoding)
    
    print(DS_feature.shape)
    print(DS_label.shape)
    
    d1 = DS_label[1,:,:]
    d2 = DS_feature[1,:,:]

    Wp = np.percentile(d1,99.9)
    Wmin = np.amin(d1)
    Wmax = np.amax(d1)
    print(Wmin, Wmax, Wp)

    Qp = np.percentile(d2, 99.9)
    Qmin = np.amin(d2)
    Qmax = np.amax(d2)
    print(Qmin, Qmax, Qp)

    cmap = plt.cm.Spectral_r

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(36,6))

    w5 = ax1.imshow(d1, cmap=cmap)
    ax1.set_title("W at 500m AGL level")
    #ax = plt.axes(projection=ccrs.PlateCarree())
    #ax.set_global()
    #ax.coastlines()
    w5.set_clim(vmin=Wmin, vmax=Wmax)
    fig.colorbar(w5, ax=ax1, orientation='vertical', label='')

    qr5 = ax2.imshow(d2, cmap=cmap)
    ax2.set_title("QRAIN at 500m AGL")
    #ax = plt.axes(projection=ccrs.PlateCarree())
    #ax.set_global()
    #ax.coastlines()
    qr5.set_clim(vmin=Qmin,vmax=Qmax)
    fig.colorbar(qr5, ax=ax2, orientation='vertical', label='')
